In [11]:
import pandas as pd
import tsfresh as tsf
from tsfresh.feature_extraction import settings
from pathlib import Path
import pycatch22
import numpy as np

import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy.stats import linregress
from scipy.fft import fft, fftfreq
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis


import warnings
warnings.filterwarnings('ignore')

## Dataset with scores

In [12]:
df = pd.read_csv('data\local_global_res_13_12.csv')

In [13]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,train_time,forecast_time,MAE,MSE,RMSE,MASE,RMSSE,MAPE,SMAPE,naming_orig,model_name,dataset_name,horizon,split,pred_time
0,0,0.0,3.812914,4.442261,18.260277,619.323792,24.886217,0.896638,0.792087,inf,16.386864,danish_atm_daily_5,CatBoostAutoRegressivePipelineEtna_3lags_gl,danish_atm_daily,30,test,NaN
1,1,0.0,3.485261,4.376619,28.395468,1160.189331,34.061552,1.088141,0.918072,41.157335,17.157184,danish_atm_daily_84,CatBoostAutoRegressivePipelineEtna_3lags_gl,danish_atm_daily,30,validation,NaN
2,2,0.0,3.812914,4.442261,12.825266,269.356567,16.412086,0.814535,0.757765,39.022604,13.038494,danish_atm_daily_32,CatBoostAutoRegressivePipelineEtna_3lags_gl,danish_atm_daily,30,test,NaN
3,3,0.0,3.485261,4.376619,26.284388,1121.348877,33.486548,0.918232,0.792925,57.078475,18.962322,danish_atm_daily_25,CatBoostAutoRegressivePipelineEtna_3lags_gl,danish_atm_daily,30,validation,NaN
4,4,0.0,3.485261,4.376619,15.145726,429.357697,20.720948,0.663444,0.597604,21.996029,10.027625,danish_atm_daily_6,CatBoostAutoRegressivePipelineEtna_3lags_gl,danish_atm_daily,30,validation,NaN


In [14]:
df['naming_orig'].value_counts()

naming_orig
mipt_alpha_193    44
mipt_alpha_342    44
mipt_alpha_510    44
mipt_alpha_151    44
mipt_alpha_584    44
                  ..
nn5_23            34
nn5_2             34
nn5_50            34
nn5_111            6
nn5_112            6
Name: count, Length: 876, dtype: int64

## Analyzing the time-series

In [15]:
input_dir = Path.cwd() / 'time_series'
folders = list(input_dir.glob("*"))


In [16]:
# Calcualtion of the initial time-series value counts

ts_full = pd.DataFrame()

for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        time_series = pd.read_csv(path)
        time_series['naming_orig'] = path.stem
        ts_full = pd.concat([ts_full, time_series])

ts_full.shape[0]

544221

All time-series has 544221 values initially

In [17]:
li1 = list(df['naming_orig'].unique())
li2 = list(ts_full['naming_orig'].unique())
temp3 = []
for element in li1:
    if element not in li2:
        temp3.append(element)
 
print(temp3)

['nn5_112', 'nn5_111']


Also `nn5_112`, `nn5_111` time-series dropped, because they presented only in dataset with scores

In [18]:
# Dropping of last 60 days

ts = pd.DataFrame()

for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        time_series = pd.read_csv(path)
        if time_series.shape[0] > 60:
            time_series = time_series[:-60]
            time_series['naming_orig'] = path.stem
            ts = pd.concat([ts, time_series])

ts.shape[0]

491866

After droping last 60 days all time-series has 491866 values

In [19]:
# Finding such time-series that has less than 60 days

for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        time_series = pd.read_csv(path)
        if time_series.shape[0] < 60:
            print(f'{path.stem} leghts: {time_series.shape[0]}')

danish_atm_daily_110 leghts: 42
danish_atm_daily_111 leghts: 28
danish_atm_daily_112 leghts: 25


This 3 dropped from the consideration, because it has less than 60 days

## Combination of all feature extraction methods together

In [20]:
def extract_time_series_features(dataset):
    # Reformat 'date' column to datetime and setting it as index
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset.set_index('date', inplace=True)
    
    # Using of 'value' column as time-series
    time_series = dataset['value']
    
    # Initialization of fetures' dictionary
    features = {}
    
    # Time fetures of time series
    features['trend_slope'], _, _, _, _ = linregress(np.arange(len(time_series)), time_series)
    acf_values = acf(time_series, nlags=24, fft=True)
    features['seasonality_strength'] = np.max(acf_values[1:])
    features['num_peaks'] = len(find_peaks(time_series)[0])
    features['diff_mean'] = time_series.diff().mean()
    features['diff_std'] = time_series.diff().std()
    zcr = ((time_series[:-1] * time_series[1:]) < 0).sum()
    features['zero_crossing_rate'] = zcr
    
    # Spectral fetures
    fft_vals = fft(time_series.to_numpy())
    fft_freq = fftfreq(len(fft_vals), d=1)
    dominant_freq = fft_freq[np.argmax(np.abs(fft_vals)**2)]
    features['dominant_freq'] = np.abs(dominant_freq)
    significant_freqs = np.sum(np.abs(fft_vals)**2 > 1e-5)
    features['significant_freqs'] = significant_freqs
    for freq in range(1, 6):
        freq_mask = (fft_freq >= freq - 0.5) & (fft_freq <= freq + 0.5)
        features[f'fft_energy_freq_{freq}'] = np.sum(np.abs(fft_vals[freq_mask])**2)
    
    # Morphological fetures
    peaks, _ = find_peaks(time_series)
    peak_distances = np.diff(peaks)
    features['mean_peak_distance'] = np.mean(peak_distances) if len(peak_distances) > 0 else 0
    features['std_peak_distance'] = np.std(peak_distances) if len(peak_distances) > 0 else 0
    peak_heights = time_series.iloc[peaks]
    features['mean_peak_height'] = peak_heights.mean()
    features['std_peak_height'] = peak_heights.std()
    
    # Additional features
    extrema = np.diff(np.sort(np.concatenate([find_peaks(time_series)[0], find_peaks(-time_series)[0]])))
    features['mean_extrema_diff'] = np.mean(extrema) if len(extrema) > 0 else 0
    features['std_extrema_diff'] = np.std(extrema) if len(extrema) > 0 else 0
    curvature = np.diff(time_series, n=2).mean()
    features['mean_curvature'] = curvature
    magnitude = np.abs(fft_vals)
    normalized_magnitude = magnitude / np.sum(magnitude)
    signal_entropy = -np.sum(normalized_magnitude * np.log(normalized_magnitude + np.finfo(float).eps))
    features['signal_entropy'] = signal_entropy
    total_energy = np.sum(np.square(time_series))
    features['total_energy'] = total_energy
    features['diff_skew'] = skew(time_series.diff().dropna())
    features['diff_kurt'] = kurtosis(time_series.diff().dropna())
    features['time_series_length'] = len(time_series)
    features['mae_diff'] = np.mean(np.abs(time_series.diff().dropna()))
    
    return features

In [21]:
ts = pd.DataFrame()
features = pd.DataFrame()

tsData = pd.read_csv('time_series\mipt_alpha\mipt_alpha_0.csv', index_col=0)
pycatch22.CO_f1ecac(list(tsData['value']))


# Initialization of the Catch22 and our own featurs dataframes
catch22_df = pd.DataFrame(columns=pycatch22.catch22_all(list(tsData['value']), catch24=True)['names'])
ord_features_df = pd.DataFrame(columns=list(extract_time_series_features(tsData).keys()))

# Calculation of Catch22 and our own features across all time-series (last 60 days did not used)
for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        time_series = pd.read_csv(path)
        if time_series.shape[0] > 60:
            time_series = time_series[:-60]
            time_series['naming_orig'] = path.stem
            ts = pd.concat([ts, time_series])
            pycatch22.CO_f1ecac(list(time_series['value']))
            catch22_df.loc[len(catch22_df.index)] = pycatch22.catch22_all(list(time_series['value']),catch24=True)['values']
            ord_features_df.loc[len(ord_features_df.index)] = list(extract_time_series_features(time_series).values())

# Calculation of tsfresh features
ts_tsf = tsf.extract_features(ts, column_id='naming_orig', 
                              column_sort="date", column_value='value').rename_axis('naming_orig').reset_index()

# Combinig of all datasets to one
features = pd.concat([ts_tsf, catch22_df, ord_features_df], axis=1)

features

Feature Extraction: 100%|██████████| 80/80 [01:24<00:00,  1.05s/it]


,naming_orig,value__variance_larger_than_standard_deviation,value__has_duplicate_max,value__has_duplicate_min,value__has_duplicate,value__sum_values,value__abs_energy,value__mean_abs_change,value__mean_change,value__mean_second_derivative_central,...,std_peak_height,mean_extrema_diff,std_extrema_diff,mean_curvature,signal_entropy,total_energy,diff_skew,diff_kurt,time_series_length,mae_diff
0,danish_atm_daily_0,1.0,0.0,1.0,1.0,35327.000000,4.805377e+06,35.180921,0.398026,-0.287129,...,53.571178,1.750000,0.807847,-0.574257,5.100711,4.805377e+06,0.141664,5.759565,305.0,35.180921
1,danish_atm_daily_1,1.0,0.0,1.0,1.0,25074.000000,3.597174e+06,35.873096,-0.015228,0.443878,...,49.935416,1.710526,0.834809,0.887755,4.574701,3.597174e+06,0.406797,3.227857,198.0,35.873096
2,danish_atm_daily_10,1.0,0.0,1.0,1.0,27172.000000,3.278432e+06,31.470395,0.319079,-0.206271,...,70.440769,1.935897,1.217814,-0.412541,5.294722,3.278432e+06,1.406318,8.212164,305.0,31.470395
3,danish_atm_daily_100,1.0,0.0,1.0,1.0,12234.000000,6.264840e+05,18.096346,-0.016611,-0.010000,...,21.974311,1.728324,0.826691,-0.020000,5.210385,6.264840e+05,-0.076666,2.179983,302.0,18.096346
4,danish_atm_daily_101,1.0,0.0,1.0,1.0,13813.000000,1.375921e+06,34.407821,-0.217877,-0.081461,...,45.951714,1.712871,0.871287,-0.162921,4.689043,1.375921e+06,0.136356,2.367263,180.0,34.407821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,nn5_95,1.0,0.0,1.0,1.0,15562.835350,3.864449e+05,7.733223,0.012809,0.006494,...,7.677542,1.766990,0.872491,0.012989,5.944508,3.864449e+05,-0.730954,1.531685,731.0,7.733223
867,nn5_96,1.0,0.0,1.0,1.0,9925.473435,1.695436e+05,5.359904,0.002342,0.004672,...,6.711614,1.647059,0.750477,0.009345,5.990598,1.695436e+05,0.087705,1.590454,731.0,5.359904
868,nn5_97,1.0,0.0,1.0,1.0,14966.333509,3.548706e+05,7.519889,0.005747,-0.001813,...,8.155347,1.615213,1.025204,-0.003626,5.929239,3.548706e+05,-0.586895,1.892068,731.0,7.519889
869,nn5_98,1.0,0.0,1.0,1.0,10153.037875,1.618989e+05,3.804162,0.010052,0.004339,...,5.250554,2.275000,1.249750,0.008677,5.920899,1.618989e+05,-0.488383,2.809081,731.0,3.804162


In [22]:
features.to_csv('All_features.csv')

## Tsfresh feature extraction separately

In [23]:
ts_tsf = tsf.extract_features(ts, column_id='naming_orig', column_sort="date", column_value='value').rename_axis('naming_orig').reset_index()

Feature Extraction: 100%|██████████| 80/80 [01:33<00:00,  1.17s/it]


In [24]:
ts_tsf.columns

Index(['naming_orig', 'value__variance_larger_than_standard_deviation',
       'value__has_duplicate_max', 'value__has_duplicate_min',
       'value__has_duplicate', 'value__sum_values', 'value__abs_energy',
       'value__mean_abs_change', 'value__mean_change',
       'value__mean_second_derivative_central',
       ...
       'value__fourier_entropy__bins_5', 'value__fourier_entropy__bins_10',
       'value__fourier_entropy__bins_100',
       'value__permutation_entropy__dimension_3__tau_1',
       'value__permutation_entropy__dimension_4__tau_1',
       'value__permutation_entropy__dimension_5__tau_1',
       'value__permutation_entropy__dimension_6__tau_1',
       'value__permutation_entropy__dimension_7__tau_1',
       'value__query_similarity_count__query_None__threshold_0.0',
       'value__mean_n_absolute_max__number_of_maxima_7'],
      dtype='object', length=784)

In [25]:
ts_tsf.to_csv('Tsfresh_features.csv')

## Catch22 feature extraction separately

In [26]:
tsData = list(pd.read_csv('time_series\mipt_alpha\mipt_alpha_0.csv')['value'])
pycatch22.CO_f1ecac(tsData)

catch22_features = pycatch22.catch22_all(tsData,catch24=True)['names']
catch22_features.append('naming_orig')
catch22_df = pd.DataFrame(columns=catch22_features)

In [27]:
for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        tsData = list(pd.read_csv(path)['value'])
        if len(tsData) > 60:
            tsData = tsData[:-60]
            pycatch22.CO_f1ecac(tsData)
            values = pycatch22.catch22_all(tsData,catch24=True)['values']
            values.append(path.stem)
            catch22_df.loc[len(catch22_df.index)] = values

In [28]:
catch22_df.columns

Index(['DN_HistogramMode_5', 'DN_HistogramMode_10', 'CO_f1ecac',
       'CO_FirstMin_ac', 'CO_HistogramAMI_even_2_5', 'CO_trev_1_num',
       'MD_hrv_classic_pnn40', 'SB_BinaryStats_mean_longstretch1',
       'SB_TransitionMatrix_3ac_sumdiagcov', 'PD_PeriodicityWang_th0_01',
       'CO_Embed2_Dist_tau_d_expfit_meandiff',
       'IN_AutoMutualInfoStats_40_gaussian_fmmi',
       'FC_LocalSimple_mean1_tauresrat', 'DN_OutlierInclude_p_001_mdrmd',
       'DN_OutlierInclude_n_001_mdrmd', 'SP_Summaries_welch_rect_area_5_1',
       'SB_BinaryStats_diff_longstretch0', 'SB_MotifThree_quantile_hh',
       'SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1',
       'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1',
       'SP_Summaries_welch_rect_centroid', 'FC_LocalSimple_mean3_stderr',
       'DN_Mean', 'DN_Spread_Std', 'naming_orig'],
      dtype='object')

In [29]:
catch22_df.to_csv('Catch22_features.csv')

## Ordinary features separately

In [30]:
def extract_time_series_features(dataset):
    # Преобразование столбца 'date' в формат datetime и установка его как индекс
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset.set_index('date', inplace=True)
    
    # Использование столбца 'value' как временного ряда
    time_series = dataset['value']
    
    # Инициализация словаря для хранения фичей
    features = {}
    
    # Временные признаки
    features['trend_slope'], _, _, _, _ = linregress(np.arange(len(time_series)), time_series)
    acf_values = acf(time_series, nlags=24, fft=True)
    features['seasonality_strength'] = np.max(acf_values[1:])
    features['num_peaks'] = len(find_peaks(time_series)[0])
    features['diff_mean'] = time_series.diff().mean()
    features['diff_std'] = time_series.diff().std()
    zcr = ((time_series[:-1] * time_series[1:]) < 0).sum()
    features['zero_crossing_rate'] = zcr
    
    # Спектральные признаки
    fft_vals = fft(time_series.to_numpy())
    fft_freq = fftfreq(len(fft_vals), d=1)
    dominant_freq = fft_freq[np.argmax(np.abs(fft_vals)**2)]
    features['dominant_freq'] = np.abs(dominant_freq)
    significant_freqs = np.sum(np.abs(fft_vals)**2 > 1e-5)
    features['significant_freqs'] = significant_freqs
    for freq in range(1, 6):
        freq_mask = (fft_freq >= freq - 0.5) & (fft_freq <= freq + 0.5)
        features[f'fft_energy_freq_{freq}'] = np.sum(np.abs(fft_vals[freq_mask])**2)
    
    # Морфологические признаки
    peaks, _ = find_peaks(time_series)
    peak_distances = np.diff(peaks)
    features['mean_peak_distance'] = np.mean(peak_distances) if len(peak_distances) > 0 else 0
    features['std_peak_distance'] = np.std(peak_distances) if len(peak_distances) > 0 else 0
    peak_heights = time_series.iloc[peaks]
    features['mean_peak_height'] = peak_heights.mean()
    features['std_peak_height'] = peak_heights.std()
    
    # Дополнительные фичи
    extrema = np.diff(np.sort(np.concatenate([find_peaks(time_series)[0], find_peaks(-time_series)[0]])))
    features['mean_extrema_diff'] = np.mean(extrema) if len(extrema) > 0 else 0
    features['std_extrema_diff'] = np.std(extrema) if len(extrema) > 0 else 0
    curvature = np.diff(time_series, n=2).mean()
    features['mean_curvature'] = curvature
    magnitude = np.abs(fft_vals)
    normalized_magnitude = magnitude / np.sum(magnitude)
    signal_entropy = -np.sum(normalized_magnitude * np.log(normalized_magnitude + np.finfo(float).eps))
    features['signal_entropy'] = signal_entropy
    total_energy = np.sum(np.square(time_series))
    features['total_energy'] = total_energy
    features['diff_skew'] = skew(time_series.diff().dropna())
    features['diff_kurt'] = kurtosis(time_series.diff().dropna())
    features['time_series_length'] = len(time_series)
    features['mae_diff'] = np.mean(np.abs(time_series.diff().dropna()))
    
    return features


In [31]:
tsData = pd.read_csv('time_series\mipt_alpha\mipt_alpha_0.csv', index_col=0)

features = list(extract_time_series_features(tsData).keys())
features.append('naming_orig')
ord_features_df = pd.DataFrame(columns=features)

In [32]:
tsData = pd.read_csv('time_series\mipt_alpha\mipt_alpha_0.csv', index_col=0)

features = list(extract_time_series_features(tsData).keys())
features.append('naming_orig')
ord_features_df = pd.DataFrame(columns=features)
ord_features_df

for folder in folders:
    paths = list(folder.glob("*"))
    for path in paths:
        tsData = pd.read_csv(path, index_col=0)
        if tsData.shape[0] > 60:
            values = list(extract_time_series_features(tsData).values())
            values.append(path.stem)
            ord_features_df.loc[len(ord_features_df.index)] = values

In [33]:
ord_features_df.columns

Index(['trend_slope', 'seasonality_strength', 'num_peaks', 'diff_mean',
       'diff_std', 'zero_crossing_rate', 'dominant_freq', 'significant_freqs',
       'fft_energy_freq_1', 'fft_energy_freq_2', 'fft_energy_freq_3',
       'fft_energy_freq_4', 'fft_energy_freq_5', 'mean_peak_distance',
       'std_peak_distance', 'mean_peak_height', 'std_peak_height',
       'mean_extrema_diff', 'std_extrema_diff', 'mean_curvature',
       'signal_entropy', 'total_energy', 'diff_skew', 'diff_kurt',
       'time_series_length', 'mae_diff', 'naming_orig'],
      dtype='object')

In [34]:
ord_features_df.to_csv('Ordinary_features.csv')